In [6]:
import pandas as pd
import nltk
import json

# download nltk corpus (first time only)
# nltk.download("all")
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
# Installiere das SnowballStemmer für die deutsche Sprache
# nltk.download('stopwords')
# nltk.download('punkt')

## Helper functions

In [10]:
# the columns are beeing merged togheter here
def merge_columns(row):
    columns = [row.get('page_title', ''), row.get('sub_title', ''),
            row.get('introduction', ''), row.get('summary_box', ''),
            row.get('content', ''), row.get('accordion', '')]
    # columns = dataframe.columns.values.tolist()
    # li = [row.get(col, '') for col in columns if col != "url"]
    url = row.get('url', '')
    result = ' '.join(filter(None, ['url'] + columns)) #filter(None, li)
    return result

In [2]:
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    filtered_tokens = [
        token for token in tokens if token not in stopwords.words("german")
    ]
    stemmer = SnowballStemmer("german")
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]
    processed_text = " ".join(stemmed_tokens)
    return processed_text

In [3]:
def get_sentiment(text_string):
    scores = analyzer.polarity_scores(text_string)
    return pd.Series(scores)

def add_sentiment_cols(dataframe, text_col="text"):
    analyzer = SentimentIntensityAnalyzer()
    sentiment_cols = dataframe[text_col].apply(get_sentiment)
    dataframe = pd.concat([dataframe, sentiment_cols], axis=1)
    dataframe['judgement'] = dataframe['compound'].apply(lambda x: 'positive' if x > 0.05 else ("negative" if x < -0.05 else 'neutral'))
    return dataframe

In [4]:
def get_analysis_data(json_file):
    pandas_df = pd.read_json(json_file)
    filtered_df = pandas_df.loc[(pandas_df.loc[:, ~pandas_df.columns.isin(['url', 'page_title'])] != "").any(axis=1)] # exkludiert alle spalten von der Analyse die "" in den spalten ausser "page_title" haben 
    data = pd.DataFrame()
    data["text"] = filtered_df.apply(merge_columns, axis=1)
    data["text"] = data["text"].apply(preprocess_text)
    return add_sentiment_cols(data)

## Execution of Code

In [9]:
display(get_analysis_data("hans.json"))

NameError: name 'analyzer' is not defined

#### Ich möchte url und page_title immer noch vor text